In [ ]:
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
# Example of working live html parser without crawler block and user-agent spoof

# url = "http://www.python.org"
# resp = requests.get(url)
# html = resp.content
# print(html)

In [ ]:
### Working example for a single KSL search results page

# Define root url for KSL cars
rooturl = "https://cars.ksl.com"

# Note the url below specifies that we're looking for 96 per page and the default sort of newest to oldest posting
# This note about newest to oldest is useful so that we can avoid scraping repeat listings based on their timestamps
url = "https://cars.ksl.com/search/newUsed/Used;Certified/perPage/96/page/0"

# Need to spoof a user-agent in order to get past crawler block
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'

# Note: The above user_agent might need to be rotated (along with IP) to avoid IP ban
# Example found on https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/

# Open live page (as opposed to downloaded)
resp = requests.get(url, headers = {'User-Agent': user_agent})
html = resp.content
pgsoup = BeautifulSoup(html)
lastpg = int(pgsoup.find(attrs={"title": "Go to last page"}).text.strip()) # Note that this is 1 more than number from href for this page
print(f'Total number of search results pages: {lastpg}')
# print('*********************')
# print()
# print(pgsoup)

print()

links = pgsoup.select("div.title > a.link") # grab all 96 (or up to 96) links
print(f'Total number of links found on current page: {len(links)}')
tstamps = pgsoup.select("div.listing-detail-line script") # grab all 96 (or up to 96) timestamps
print(f'Total number of timestamps found on current page: {len(tstamps)}')

print()

# for tstamp in tstamps:
#     print(int(re.search('(\d+)',tstamp.text).group(0))) # <-- This is WORKING code to extract timestamp for each listing from search page
# print()

maxresults = 15
print(f'Limiting Subsequent Listing Results to {maxresults}')
print()

# Loop through links and scrape data for each new listing
for link in links[:maxresults]: # *** only load first x results for now to avoid ban before implementing spoofing
    # We're going to want to strip the "?ad_cid=[number]" from the end of these links as they're not needed to load the page properly
    # Regular expressions should come in handy here
    
    cutidx = re.search('(\?ad_cid=.+)',link['href']).start()
    currlink = link['href'][:cutidx]
       
    # Somewhere here we should do a check to make sure that the timestamp for currlink is newer than our newest file in our repository
    # That is, compare the timestamps with a simple conditional, where if the conditional is not met, this loop breaks to avoid useless computation time
    
    # Open listing link and pull html from it
    fulllink = '/'.join([rooturl.rstrip('/'), currlink.lstrip('/')])
    
    resp = requests.get(fulllink, headers = {'User-Agent': user_agent})
    lsthtml = resp.content
    lstsoup = BeautifulSoup(lsthtml)
    
    name = lstsoup.select('h1.title > div')[0].text.strip() # not really useful for the project, likely, but just for this working example
    price = lstsoup.select('h3.price')[0].text.strip()
    
    print(f'Name: {name}\nPrice: {price}')
    print()
    
    specs = lstsoup.select('ul.listing-specifications')
    
    # The above specs are going to need to hacky scraping to tease out values such as year, make, model, etc.
    